# Chapter 5 — PyTorch Essentials

This notebook mirrors Chapter 5. It’s self‑contained and can run on CPU, Apple MPS, or CUDA.
If running on Google Colab, the first cell installs PyTorch.

In [ ]:
# Install PyTorch in Colab or when torch is missing.
try:
    import torch  # noqa:F401
    print('torch:', torch.__version__)
except Exception:
    import os
    gpu = os.system('nvidia-smi > /dev/null 2>&1') == 0
    index = 'https://download.pytorch.org/whl/cu121' if gpu else 'https://download.pytorch.org/whl/cpu'
    get_ipython().run_line_magic('pip', f'install -q torch --index-url {index}')
    import torch
    print('torch:', torch.__version__, 'cuda?', torch.cuda.is_available())


In [ ]:
# Pick device
import torch
def pick_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    mps = getattr(torch.backends, 'mps', None)
    if mps and torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')
device = pick_device()
device


## Tensors and Shapes

In [ ]:
x = torch.arange(6, dtype=torch.float32).reshape(2, 3)
y = torch.ones_like(x)
x.shape, y.shape, x.device


In [ ]:
b = torch.tensor([10.0, 20.0, 30.0])
(x + b).shape, (x + b)[0]


In [ ]:
x = x.to(device)
x.device


## Autograd in a Nutshell

In [ ]:
w = torch.tensor([2.0, -3.0, 0.5], requires_grad=True)
v = torch.tensor([1.0, 2.0, 3.0])
loss = (w * v).sum()
loss.backward()
w.grad


## Optimizers and Parameters

In [ ]:
model = torch.nn.Linear(3, 1).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=3e-3)
crit = torch.nn.MSELoss()
type(model), type(opt), type(crit)


## Training Loop Pattern

In [ ]:
# CPU generator for determinism
g_cpu = torch.Generator().manual_seed(0)
g_cpu


In [ ]:
# Create inputs X
X = torch.randn(64, 3, generator=g_cpu).to(device)
X


In [ ]:
# Ground-truth weights
true_w = torch.tensor([1.0, -2.0, 0.5], device=device)
true_w


In [ ]:
# Targets with a touch of noise
y = (X @ true_w) + 0.1 * torch.randn(64, generator=g_cpu).to(device)
y[:8]


In [ ]:
# Model, optimizer, loss
model = torch.nn.Linear(3, 1).to(device)
model


In [ ]:
opt = torch.optim.AdamW(model.parameters(), lr=5e-3)
opt


In [ ]:
loss_fn = torch.nn.MSELoss()
loss_fn


In [ ]:
# Train
losses = []
for step in range(201):
    opt.zero_grad()
    pred = model(X).squeeze(-1)
    loss = loss_fn(pred, y)
    loss.backward()
    opt.step()
    losses.append(loss.detach().item())
    if step % 50 == 0:
        print(step, round(losses[-1], 4))
losses[-1]


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'
# Plot loss vs step
plt.figure(figsize=(4.5,3))
plt.plot(losses, label='train loss')
plt.xlabel('step')
plt.ylabel('MSE')
plt.title('Linear regression: loss vs. step')
plt.legend(); plt.tight_layout()


## Linear Regression (End‑to‑End Example)

In [ ]:
def linreg_demo(epochs: int = 400, lr: float = 3e-2):
    g_cpu = torch.Generator().manual_seed(42)
    w_true = torch.tensor([2.0, -3.5], device=device)
    b_true = torch.tensor(0.5, device=device)
    X = torch.randn(128, 2, generator=g_cpu).to(device)
    y = (X @ w_true) + b_true + 0.1 * torch.randn(128, generator=g_cpu).to(device)
    model = torch.nn.Linear(2, 1).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr)
    loss_fn = torch.nn.MSELoss()
    for step in range(epochs + 1):
        opt.zero_grad()
        pred = model(X).squeeze(-1)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        if step % 100 == 0:
            print(step, round(loss.item(), 4))
    return model.weight.detach().squeeze(0), model.bias.detach().squeeze(0)

w, b = linreg_demo()
w.cpu().tolist(), float(b)
